In [19]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [2]:
# Load the csv file into a dataframe
file = "Resources/database.csv"
df = pd.read_csv(file)
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [3]:
# Print a list of the columns
df.columns

Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [4]:
# Delete unneccesary columns
# Create a list to hold all the unneccesary columns
extra = ['Time', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status']

df = df.drop(extra, axis=1)

df.head()

,Date,Latitude,Longitude,Magnitude
0,01/02/1965,19.246,145.616,6.0
1,01/04/1965,1.863,127.352,5.8
2,01/05/1965,-20.579,-173.972,6.2
3,01/08/1965,-59.076,-23.557,5.8
4,01/09/1965,11.938,126.427,5.8


In [14]:
# Filter the data so that only California earthquakes are shown
# Minimum latitude and longitude are 32, -124
# Max latitude and longitude are 42, -114

df_lat_32 = df.loc[df["Latitude"] >= 32 ]
df_lat_42 = df_lat_32.loc[df["Latitude"]<= 42]
df_long_124 = df_lat_42.loc[df_lat_42["Longitude"]>= -124]
df_long_114 = df_long_124.loc[df_long_124["Longitude"]<= -114]
len(df_long_114)

116

In [16]:
# Add a column to say the state
df_long_114['State'] = "CA"
df_long_114.head()

C:\Users\18587\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date,Latitude,Longitude,Magnitude,State
498,08/16/1966,37.380000,-114.157000,5.70,CA
522,09/12/1966,39.385000,-120.252000,5.90,CA
565,12/20/1966,37.302167,-116.408333,5.62,CA
890,04/09/1968,33.179833,-116.103000,6.60,CA
897,04/26/1968,37.295333,-116.455667,5.63,CA


In [17]:
# Drop the latitude and longitude columns
new_df = df_long_114.drop(['Latitude', 'Longitude'], axis=1)
new_df.head()

,Date,Magnitude,State
498,08/16/1966,5.70,CA
522,09/12/1966,5.90,CA
565,12/20/1966,5.62,CA
890,04/09/1968,6.60,CA
897,04/26/1968,5.63,CA


In [18]:
# Rename the columns to allign with the table in Postgres
earthquake_df = new_df.rename(columns={'Date': 'date', 'Magnitude': 'magnitude', 'State': 'epicenter'})
earthquake_df.head()

,date,magnitude,epicenter
498,08/16/1966,5.70,CA
522,09/12/1966,5.90,CA
565,12/20/1966,5.62,CA
890,04/09/1968,6.60,CA
897,04/26/1968,5.63,CA


In [20]:
# Connect to the database
connection_string = f"postgres:{password}@localhost:5432/earthquakes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# Check table names
engine.table_names()

['cali_earthquakes']

In [24]:
# Load the dataframe into the database
earthquake_df.to_sql(name='cali_earthquakes', con=engine, if_exists='append', index=False)

In [25]:
# Confirm that the data has been added
pd.read_sql_query('select * from cali_earthquakes', con=engine).head()

,date,magnitude,epicenter
0,08/16/1966,5.70,CA
1,09/12/1966,5.90,CA
2,12/20/1966,5.62,CA
3,04/09/1968,6.60,CA
4,04/26/1968,5.63,CA
